In [1]:
import os
os.chdir('../')
%pwd

'd:\\Data Science\\Udemy MLOPs\\DataScienceProject'

In [9]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelTrainerConfig:
    root_dir:Path
    train_data_path: Path
    test_data_path: Path
    model_name:str
    target_column:str
    alpha: float
    l1_ratio: float



In [10]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [11]:
#Configuraion manager
class ConfigurationManger:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name
        )
        return model_trainer_config

In [12]:
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib
from src.datascience import logger

In [15]:
## Component
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config=config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        text_x = test_data.drop([self.config.target_column], axis=1)
        train_y=train_data[self.config.target_column]
        test_y=test_data[self.config.target_column]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        logger.info("model training successful")

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))




In [16]:
try:
    config=ConfigurationManger()
    model_trainer_config= config.get_model_trainer_config()
    model_trainer=ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-02-15 15:15:17,419: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-15 15:15:17,442: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-15 15:15:17,492: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-15 15:15:17,507: INFO: common: created directory at: artifacts]
[2025-02-15 15:15:17,519: INFO: common: created directory at: artifacts/model_trainer]


[2025-02-15 15:15:18,087: INFO: 1202138275: model training successful]
